In [1]:
import pandas as pd
import numpy as np
from WindPy import w

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
# fname = '\\\\hfm-pubshare\HFM各部门共享\Harvest嘉实全体员工\YANJJ\综合信息查询_成交回报.xls'
fname = '../data/综合信息查询_成交回报.xls'

In [4]:
df = pd.read_excel(fname, index_col=0, converters={'证券代码': str})

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [5]:
df = df[df['委托方向'] == '卖出']

In [6]:
df.loc[df.index, '发生日期'] = pd.to_datetime(df['发生日期'])

In [7]:
def to_windcode(s):
    if s.startswith('6'):
        return s + '.SH'
    else:
        return s + '.SZ'

In [8]:
df['windcode'] = [to_windcode(x) for x in df['证券代码']]

In [9]:
data = w.wss(df['windcode'].drop_duplicates().tolist(), 'ipo_date,mkt,ipo_price')

In [10]:
ipo_date = {k: v for k, v in zip(data.Codes, data.Data[0])}
mkt = {k: v for k, v in zip(data.Codes, data.Data[1])}
ipo_price = {k: v for k, v in zip(data.Codes, data.Data[2])}

In [11]:
df['ipo_date'] = pd.to_datetime([ipo_date[x] for x in df['windcode']])
df['mkt'] = [mkt[x] for x in df['windcode']]
df['ipo_price'] = [ipo_price[x] for x in df['windcode']]

In [12]:
df['days'] = df['发生日期'] - df['ipo_date']

In [13]:
df['days'] = [x.days for x in df['days']]

# 新股判断条件

In [14]:
df = df[df['days'] < 45]

In [15]:
df.head()

,发生日期,基金编号,基金名称,组合名称,证券代码,证券名称,委托方向,指令数量,持仓,最新价,...,今开盘价,是否强平委托,代下达人,是否强平,平仓盈亏,windcode,ipo_date,mkt,ipo_price,days
序号,,,,,,,,,,,,,,,,,,,,,
1,2020-10-09,418.0,嘉实创新成长混合,缺省组合,605099,共创草坪,卖出,624,0.0,24.45,...,0.0,非强平,闫晶晶,否,0,605099.SH,2020-09-30,主板,15.44,9
2,2020-10-09,418.0,嘉实创新成长混合,缺省组合,605218,伟时电子,卖出,837,0.0,18.03,...,0.0,非强平,闫晶晶,否,0,605218.SH,2020-09-28,主板,10.97,11
3,2020-10-09,418.0,嘉实创新成长混合,缺省组合,688093,世华科技,卖出,4595,0.0,27.40,...,0.0,非强平,闫晶晶,否,0,688093.SH,2020-09-30,科创板,17.55,9
4,2020-10-09,418.0,嘉实创新成长混合,缺省组合,688585,上纬新材,卖出,4922,0.0,14.51,...,0.0,非强平,闫晶晶,否,0,688585.SH,2020-09-28,科创板,2.49,11
5,2020-10-09,418.0,嘉实创新成长混合,缺省组合,688595,芯海科技,卖出,3190,0.0,62.94,...,0.0,非强平,闫晶晶,否,0,688595.SH,2020-09-28,科创板,22.82,11


# 创新成长打新统计

In [16]:
results = pd.DataFrame(columns=['基金名称', '基金公司名称', 
                                '是否全部卖出（科）', '原持有股数（科）', 
                                '已卖出股数（科）', '卖出持股比例（科）',
                                '科创板股票成本金额', '科创板打新收益',
                                '是否全部卖出（创）', '原持有股数（创）',
                                '已卖出股数（创）', '卖出持股比例（创）',
                                '创业板股票成本金额', '创业板打新收益',
                                '主板打新收益', '打新总收益', '备注'], index=[1])

In [17]:
results.loc[1, '基金名称'] = '嘉实创新成长混合'
results.loc[1, '基金公司名称'] = '嘉实基金管理有限公司'

### 科创板

In [18]:
star = df[df['mkt'] == '科创板']

In [19]:
results.loc[1, '原持有股数（科）'] = (star['成交数量'] + star['持仓']).sum() / 1e4
results.loc[1, '已卖出股数（科）'] = star['成交数量'].sum() / 1e4
results.loc[1, '卖出持股比例（科）'] = results.loc[1, '已卖出股数（科）'] / results.loc[1, '原持有股数（科）']
results.loc[1, '是否全部卖出（科）'] = '是' if results.loc[1, '卖出持股比例（科）'] == 1 else '否'

In [20]:
results.loc[1, '科创板股票成本金额'] = (star['ipo_price']*star['成交数量']).sum()
results.loc[1, '科创板打新收益'] = (star['成交金额']-star['ipo_price']*star['成交数量']).sum()

In [21]:
results

,基金名称,基金公司名称,是否全部卖出（科）,原持有股数（科）,已卖出股数（科）,卖出持股比例（科）,科创板股票成本金额,科创板打新收益,是否全部卖出（创）,原持有股数（创）,已卖出股数（创）,卖出持股比例（创）,创业板股票成本金额,创业板打新收益,主板打新收益,打新总收益,备注
1,嘉实创新成长混合,嘉实基金管理有限公司,是,1.4213,1.4213,1,298568,284131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 创业板

In [22]:
entr = df[df['mkt'] == '创业板']

In [23]:
results.loc[1, '原持有股数（创）'] = (entr['成交数量'] + entr['持仓']).sum() / 1e4
results.loc[1, '已卖出股数（创）'] = entr['成交数量'].sum() / 1e4
results.loc[1, '卖出持股比例（创）'] = results.loc[1, '已卖出股数（创）'] / results.loc[1, '原持有股数（创）']
results.loc[1, '是否全部卖出（创）'] = '是' if results.loc[1, '卖出持股比例（创）'] == 1 else '否'

In [24]:
results.loc[1, '创业板股票成本金额'] = (entr['ipo_price']*entr['成交数量']).sum()
results.loc[1, '创业板打新收益'] = (entr['成交金额']-entr['ipo_price']*entr['成交数量']).sum()

In [25]:
results

,基金名称,基金公司名称,是否全部卖出（科）,原持有股数（科）,已卖出股数（科）,卖出持股比例（科）,科创板股票成本金额,科创板打新收益,是否全部卖出（创）,原持有股数（创）,已卖出股数（创）,卖出持股比例（创）,创业板股票成本金额,创业板打新收益,主板打新收益,打新总收益,备注
1,嘉实创新成长混合,嘉实基金管理有限公司,是,1.4213,1.4213,1,298568,284131,否,3.2894,3.0563,0.929136,557620,1.39699e+06,NaN,NaN,NaN


### 非科创板

In [26]:
main = df[df['mkt'] == '主板']

In [27]:
results.loc[1, '主板打新收益'] = (main['成交金额']-main['ipo_price']*main['成交数量']).sum()

In [28]:
results.loc[1, '打新总收益'] = (df['成交金额'] - df['ipo_price']*df['成交数量']).sum()

In [29]:
results.to_excel('../data/打新统计.xlsx')